In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv

NU_FIDUCIAL = 70.0e6 # 70 MHz
DURATION = 3600. # 3600 seconds
kB = 1.38e-16 # in cgs
cc = 2.998e10  # in cgs
c = 2.998e8 # m/s

# Radio Detectability of Hot Jupiters
For instruments GMRT, HERA, LOFAR, SKA, UTR-2, VLA, we want to estimate Sensitivity, Time, Resolution, Confusion Limit

The radiometer equation:
$$
\frac{\sigma}{1 \rm~Jy} = 10^{23} \frac{2 k}{\lambda^2} \frac{T_{\rm sys} \Omega_{\rm station}}{\sqrt{2 B t N^2/2}}
$$

where $B$ is the bandwidth, $t$ is the observing time, and $N$ is the number of antennas.

or, since 
$$\Omega \approx (\lambda / D)^2$$
we have
$$
\frac{\sigma}{1 \rm~Jy} \approx 
10^{23} \frac{2 k}{D^2} \frac{T_{\rm sys}}{\sqrt{2 B t N^2/2}}
$$

Note that the beam is actually
$$\Omega = \pi (1.22 \lambda / 2 D)^2 / \ln(2)$$
so the equation is
$$
\frac{\sigma}{1 \rm~Jy} = 
10^{23} \frac{2 k \, 1.22^2 \, \pi}{4 \ln{(2)} D^2} \frac{T_{\rm sys}}{\sqrt{2 B t N^2/2}}
$$
which is larger than the approximation by a factor of 1.69.

Galactic synchrotron dominates below 100MHz (Di Matteo+ 2002, Santos, Cooray, Knox 2005, McQuinn+ 2006).

Tsys from Rogers & Bowman (2008)  
Tsys = Tsky = 237K(nu/150MHz)**-2.52

In the 100-200~MHz range, their fit (equation 10) is:
$$
T_{\rm sky} = T_{150} \left( \frac{\nu}{150~\rm MHz}\right)^{-\beta} + T_{\rm CMB} 
$$
where $T_{150} = 283.2~\rm K$, $T_{\rm CMB} = 2.73~\rm K$, and $\beta=2.47$.  
$T_{\rm sys}$ is then the sum of the receiver noise temperature $T_{\rm rx}\gtrsim 60~\rm K$
and $T_{\rm sky}$.
$$
T_{\rm sys} = T_{\rm rx} + T_{\rm sky} 
$$


In [2]:
def rms_uJy(observatory, tt):
    """
    Returns the RMS noise in Jy for an observatory for a certain time and frequency.
    
    Default frequency is 70 MHz.
    """
    nn = observatory['num_antennas']
    num_baselines = nn * (nn - 1.) / 2.
    nu = observatory['freq']*1.0e6 
#    Tsky = 237 * (nu / 150e6)**(-2.52)
    Trx = observatory['T_rx']
    Tcmb = 2.73
    Tsky = 283.2 * (nu / 150e6)**(-2.47) + Tcmb
    Tsys = Tsky + Trx
    lam = cc / nu
    bb = observatory['bandwidth_MHz'] * 1.0e6 # convert MHz to Hz
    DD = observatory['dish_diam_m'] * 100. # convert meters to cm
    area_eff = 0.7*np.pi * (DD/2.)**2
    # the prefactor contains Jy in cgs, 2 polarizations, the beam size from above,
    # boltzman constant, 
    prefactor = 1e23 * 2. * 1.22**2 * np.pi / (4 *np.log(2)) * kB / area_eff * 1e6
    return prefactor * Tsys / np.sqrt(2 * bb * tt * num_baselines)

In [3]:
# get table data
observatory_data = read_csv('telescope_data.csv')
observatory_data

,telescope,dish_diam_m,max_baseline_m,num_antennas,freq,bandwidth_MHz,T_rx
0,gmrt,45.0,25000,30,150,50,200
1,hera,14.0,1000,350,60,50,260
2,lofar_lba,35.0,100000,44,60,50,500
3,lofar_hba,25.0,100000,44,150,90,60
4,lwa,2.0,100000,256,60,50,260
5,mwa,4.0,1000,128,60,50,260
6,ska-low,2.4,65000,130000,125,150,60
7,ska-low-full,2.4,65000,130000,200,300,60
8,vla-4band,25.0,36000,29,60,50,260
9,vla-lobo,25.0,36000,29,200,300,60


In [4]:
# Add a column to observatory_data
# observatory_data['rms_uJy'] = np.nan * observatory_data.num_antennas

# We'll do the calculation assuming 70 MHz and 1 hr, with a bandwidth of 50 MHz
# that's a wavelength of 4.3 meters
cc_mks = cc / 100. # in mks
observatories = {}
for ii, observatory in enumerate(observatory_data.telescope):
    this_observatory = {}
    for this_property in observatory_data:
        if this_property == 'telescope':
            continue
        this_observatory[this_property] = observatory_data[this_property].iloc[ii]
    bmax = this_observatory['max_baseline_m'] # in m
    this_nu = this_observatory['freq']*1.0e6
    theta = cc_mks / (this_nu * bmax)
    theta_asec = theta * 180. * 3600. / np.pi
    observatory_data.loc[ii, 'theta_res_asec'] = round(theta_asec,2)
    observatory_data.loc[ii, 'rms_uJy'] = round(rms_uJy(this_observatory, DURATION),2)
    observatory_data.loc[ii, 'Tint_20uJy'] = round((observatory_data.loc[ii, 'rms_uJy']/20.)**2,2)
    observatory_data.loc[ii, 'Tint_80uJy'] = round((observatory_data.loc[ii, 'rms_uJy']/80.)**2,2)
    observatories[observatory] = this_observatory
#observatories
observatory_data

,telescope,dish_diam_m,max_baseline_m,num_antennas,freq,bandwidth_MHz,T_rx,theta_res_asec,rms_uJy,Tint_20uJy,Tint_80uJy
0,gmrt,45.0,25000,30,150,50,200,16.49,162.35,65.89,4.12
1,hera,14.0,1000,350,60,50,260,1030.64,869.72,1891.03,118.19
2,lofar_lba,35.0,100000,44,60,50,500,10.31,1207.99,3648.10,228.01
3,lofar_hba,25.0,100000,44,150,90,60,4.12,189.27,89.56,5.60
4,lwa,2.0,100000,256,60,50,260,10.31,58294.91,8495741.33,530983.83
5,mwa,4.0,1000,128,60,50,260,1030.64,29204.78,2132297.94,133268.62
6,ska-low,2.4,65000,130000,125,150,60,7.61,7.80,0.15,0.01
7,ska-low-full,2.4,65000,130000,200,300,60,4.76,2.20,0.01,0.00
8,vla-4band,25.0,36000,29,60,50,260,28.63,3345.20,27975.91,1748.49
9,vla-lobo,25.0,36000,29,200,300,60,8.59,92.35,21.32,1.33


# Confusion noise
Want 1 source per 10 beams (pixels)  
Perley & Ericson (1974)  
Wieringa (1991)

$$
\sigma^2 = \int_\Omega \int_0^{5 \sigma / P} s^2 n[s] B[\Omega]^2 P[\Omega]^2 ds d\Omega
$$

$s$: source flux  
$n$: # count at $s$   -- $k s^{-1.65}$ (close to 3/2) ... and $k \propto \nu^{-0.7}$  
$B$: synthesized beam  
$P$: primary beam  
$ds$: sources  
$d\Omega$: solid angle

Remove all sources above 5$\sigma$

At 150 MHz with MWA, for $P \sim 30^\circ$, $B \sim 3.4'$, we get $\sigma \sim 0.1 \rm~Jy$.

$b_{\rm max}$ is the maximum baseline

Should scale as
$$
\sigma^2 \propto \left( \frac{\lambda}{D} \right)^2  \left( \frac{\lambda}{b_{\rm max}} \right)^2 \, ,
$$
or
$$
\sigma \propto \lambda^2 ,
$$

so at 70 MHz should be a factor of 4 worse.

So,

$$
5\sigma \sim 0.1 {\rm~Jy} \times \left( \frac{\nu}{150 \rm~MHz} \right)^{-2} \left( \frac{D}{4 \rm~m} \right)^{-2} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2}
$$

In the dense regime, the $D$ drops out.  In the sparse regime, the $D$ remains.

According to Jim Condon and Scott Ransom's upcoming book <i> Essential Radio Astronomy</i>, the confusion noise
changes between the low resolution regime ($\theta>10$ arcsec), and the resolved source counts ($\theta<10$ arcsec) regime:

$$
\sigma \approx 0.2 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( \frac{\theta}{1^{\prime}} \right)^2
~~~~~~ (\theta > 10^{\prime\prime})
$$
$$
\sigma \approx 2.2 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( \frac{\theta}{1^{\prime}} \right)^{10/3} 
~~~~~~ (\theta < 10^{\prime\prime})
$$

TODO:  
replace $\theta$ with $\lambda / D$ and express in $\nu$ and $D$, and multiply by 5 for crude estimate of $5\sigma$.  
Wait, should it actually be $\lambda / b_{\rm max}$??  
I'm going to go with that.

\begin{eqnarray}
5 \sigma & \sim {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( \frac{\lambda / b_{\rm max}}{1^{\prime}} \frac{60^{\prime}}{1^\circ} \frac{180^\circ}{\pi \rm~radians} \right)^2 \\\
 & = 1.18 \times 10^7 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( \frac{\lambda}{b_{\rm max}} \right)^2 \\\
 & = 1.18 \times 10^7 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( \lambda \right)^2 \left( b_{\rm max} \right)^{-2} \\\
 & = 1.18 \times 10^7 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( c / \nu \right)^2 \left( b_{\rm max} \right)^{-2} \\\
 & = 1.06 \times 10^{24} {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} {\rm~m^2~s^{-2}} \left( \nu \right)^{-2} \left( b_{\rm max} \right)^{-2} \\\
 & = 1.06 \times 10^{24} {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} {\rm~m^2~s^{-2}} \left(10^9 \rm~Hz \right)^{-2} \left( \frac{\nu}{1 \rm~GHz} \right)^{-2} \left( b_{\rm max} \right)^{-2} \\\
 & = 1.06 \times 10^6 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} {\rm~m^2} \left( \frac{\nu}{1 \rm~GHz} \right)^{-2} \left( b_{\rm max} \right)^{-2} \\\
 & = 1.06 \times 10^6 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-2.7} {\rm~m^2} ({1000 \rm~m})^{-2} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
 & = 1.06 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-2.7} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
 & = 178 {\rm~mJy} \left( \frac{\nu}{150 \rm~MHz} \right)^{-2.7} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
 & = 1400 {\rm~mJy} \left( \frac{\nu}{70 \rm~MHz} \right)^{-2.7} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
\end{eqnarray}

At low frequency, the dense limit applies to most/all of our possible instruments:
GMRT, HERA, LOFAR, MWA, SKA, VLA, LWA

configuration A for VLA to maximize resolution.

The approximation seems to come from the Condon et al. 1998 (NVSS paper), which has a confusion limit of 0.011 mJy in a 17.5'' beam and a survey frequency of 1420 MHz (21 cm).  Thus the equation is:
$$
\sigma_c \approx 0.011 {\rm~mJy} \left( \frac{\nu}{1420 \rm~MHz} \right)^{-0.7} \left( \frac{\theta}{17.5''} \right)^2
$$


In [5]:
def confusion_uJy(observatory, nu):
    bmax = observatory['max_baseline_m'] # in m
    answer_mJy = 1393./5.0 * (nu / 70e6)**(-2.7) * (bmax / 1000)**(-2)
    answer_uJy = answer_mJy * 1000.
    return answer_uJy

In [6]:
def confusion_uJy_highres(observatory, nu):
    bmax = observatory['max_baseline_m'] # in m
    theta = c/(nu*bmax)
    theta_amin = theta * 180. * 60. / np.pi
    answer_mJy = 2.2 * (nu / 1000e6)**(-0.7) * (theta_amin)**(10./3.)
    answer_uJy = answer_mJy * 1000.
    return answer_uJy

In [7]:
#observatory_data['conf_mJy_per_beam'] = np.nan * observatory_data.num_antennas
for ii, observatory in enumerate(observatory_data.telescope):
    this_observatory = {}
    for this_property in observatory_data:
        if this_property == 'telescope':
            continue
        this_observatory[this_property] = observatory_data[this_property].iloc[ii]
    this_nu = this_observatory['freq']*1.0e6
    if this_observatory['theta_res_asec']>10:
            observatory_data.loc[ii,'conf_uJy_per_beam'] = round(
                confusion_uJy(this_observatory,nu=this_nu),3)
    else:
            observatory_data.loc[ii,'conf_uJy_per_beam'] = round(
                confusion_uJy_highres(this_observatory,nu=this_nu),3)
observatory_data

,telescope,dish_diam_m,max_baseline_m,num_antennas,freq,bandwidth_MHz,T_rx,theta_res_asec,rms_uJy,Tint_20uJy,Tint_80uJy,conf_uJy_per_beam
0,gmrt,45.0,25000,30,150,50,200,16.49,162.35,65.89,4.12,56.940
1,hera,14.0,1000,350,60,50,260,1030.64,869.72,1891.03,118.19,422413.164
2,lofar_lba,35.0,100000,44,60,50,500,10.31,1207.99,3648.10,228.01,42.241
3,lofar_hba,25.0,100000,44,150,90,60,4.12,189.27,89.56,5.60,1.103
4,lwa,2.0,100000,256,60,50,260,10.31,58294.91,8495741.33,530983.83,42.241
5,mwa,4.0,1000,128,60,50,260,1030.64,29204.78,2132297.94,133268.62,422413.164
6,ska-low,2.4,65000,130000,125,150,60,7.61,7.80,0.15,0.01,9.672
7,ska-low-full,2.4,65000,130000,200,300,60,4.76,2.20,0.01,0.00,1.453
8,vla-4band,25.0,36000,29,60,50,260,28.63,3345.20,27975.91,1748.49,325.936
9,vla-lobo,25.0,36000,29,200,300,60,8.59,92.35,21.32,1.33,10.414
